In [32]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/usnewshealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [33]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,586278450392133633,Thu Apr 09 21:24:09 +0000 2015,Planning to hire a personal trainer? Read thes...
1,586260156155043843,Thu Apr 09 20:11:28 +0000 2015,RT @AnnaMedaris: Any dads out their who strugg...
2,586248551811932160,Thu Apr 09 19:25:21 +0000 2015,America's problem with diabetes in one map: ht...
3,586229697165586432,Thu Apr 09 18:10:26 +0000 2015,Think water &amp; fiber will cure your constip...
4,586215972731822080,Thu Apr 09 17:15:53 +0000 2015,"About to lose it? Here, try one of these offic..."


In [34]:
# -------------------------------------------------
frame.shape

(1395, 3)

In [35]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Planning to hire a personal trainer? Read these 7 tips first: http://ow.ly/LpxFq'

In [36]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://ow.ly/LpxFq
http://ow.ly/LoXQG
http://ow.ly/LoBLH
http://ow.ly/LoBkH
http://ow.ly/Loypz
http://ow.ly/Lownl
http://ow.ly/Lod4r
http://ow.ly/Lo7Eu
http://ow.ly/LmFLs
http://ow.ly/LmFzN
http://ow.ly/LmFle
http://ow.ly/LmEMh
http://ow.ly/LmsCH
http://ow.ly/Llrhj
http://ow.ly/LllTo
http://ow.ly/Lllir
http://ow.ly/LljLG
http://ow.ly/LliwN
http://ow.ly/Llh69
http://ow.ly/LjDmu
http://ow.ly/LjCNT
http://ow.ly/LjD5e
http://ow.ly/LjcXE
http://ow.ly/Ljc4A
http://ow.ly/LjbHS
http://ow.ly/Lj9F9
http://ow.ly/Lione
http://bit.ly/1FbBWkr
http://ow.ly/Libq2
http://ow.ly/LiaXg
http://bit.ly/1HLGN9s
http://ow.ly/LialD
http://ow.ly/Li9AE
http://ow.ly/LgosK
http://t.usnews.com/Zbzf4
http://ow.ly/LgnYY
http://ow.ly/LgnFC
http://ow.ly/Lgnrb
http://ow.ly/Lgglk
http://ow.ly/LfJZH
http://ow.ly/LfDFT
http://ow.ly/LfrLt
http://ow.ly/LfoPD
http://ow.ly/LfjRb
http://t.usnews.com/Z875by
http://ow.ly/L9mJK
http://ow.ly/L9lnT
http://ow.ly/L9ldY
http://ow.ly/L9kAg
http://ow.ly/L9mOm
http://ow.ly/L9VT1
http://ow.

http://ow.ly/KMKhk
What are the best foods to fight inflammation? http://ow.ly/KKVVS


http://ow.ly/KKVVS
Don't want a "diet" but just to eat healthy? Try these top-rated plans: http://ow.ly/KKVNy


http://ow.ly/KKVNy
Diagnosed with #OvarianCancer? Ask these 12 questions: http://ow.ly/KKxnA #AngelinaJolie


http://ow.ly/KKxnA
RT @alzassociation: Thank you @leonardkl for your important story sharing the latest #Alzheimers facts: http://bit.ly/1HwKMq8 #ENDALZ


http://bit.ly/1HwKMq8
How #depression warps your sense of time: http://ow.ly/KKwao via @HealthyLiving 


http://ow.ly/KKwao
Signed up for a #5K? Hit the weight room, first, @ThriveFit says: http://ow.ly/KKvSr #runchat


http://ow.ly/KKvSr
Can #acupuncture be part of an effective treatment for #infertility? http://ow.ly/KKv04 #womenshealth


http://ow.ly/KKv04
What are the best diets for #diabetes? @biggestlosernbc diet &amp; #DASH tie for 1st. http://ow.ly/KJx75


http://ow.ly/KJx75
Job forecast sunny for #PhysicianAssistants: htt


http://ow.ly/ITaCg
Nearly half of couples' #infertility problems involve the man. http://ow.ly/ITqiY


http://ow.ly/ITqiY
It's not just you: WAY more people are doing #yoga these days, a survey finds. http://ow.ly/ITqUI


http://ow.ly/ITqUI
RT @lisaespo: Before you have that #angioplasty, read this by @StevenSternberg in @usnews  http://bit.ly/1Ch5s1m


http://bit.ly/1Ch5s1m
Got indoor #allergies? Here's the moldy truth about 'em. http://ow.ly/ITp76


http://ow.ly/ITp76
Embrace the cold, dark winter – and embrace #yoga, @yogawithjake says: http://ow.ly/IT7K0


http://ow.ly/IT7K0
The Anatomy of a Broken Heart: http://ow.ly/IT5MN  #heartbreak #valentinesday 


http://ow.ly/IT5MN
EXCLUSIVE: Are doctors performing too many angioplasties? You be the judge: http://ow.ly/ISV6s


http://ow.ly/ISV6s
10 Ways Nutritionists Enjoy Chocolate by @KeriGans http://ow.ly/ISSz2


http://ow.ly/ISSz2
How to Be More Mindful at Work: http://ow.ly/IQhRb #mindfulness #careeradvice


http://ow.ly/IQhRb
How muc

http://ow.ly/H3MHA
Weather got you down? Cheer up! 10 ways to break a bad mood: http://ow.ly/H3kO8 #happiness


http://ow.ly/H3kO8
What is thought to be the oldest condiment on the planet? (Hint: It's a seed.) http://ow.ly/H04Aq


http://ow.ly/H04Aq
Is there anything #yoga's *not* good for? The latest: Yoga for #grief, by @AnnaMedaris http://ow.ly/H062E


http://ow.ly/H062E
What 4 types of calories should dominate your daily diet? @KristinKirkpat answers: http://ow.ly/H016e #nutrition


http://ow.ly/H016e
Has your relationship with your fitness tracker gone sour? Bring back that loving feeling with these 4 tips: http://ow.ly/H08gz


http://ow.ly/H08gz
Does this calorie make me look fat? Maybe. How to choose your calories wisely: http://ow.ly/H00Mz #diet #nutrition


http://ow.ly/H00Mz
Back pain? You're not alone. @lisaespo has 11 ways to cope: http://ow.ly/H06Aj


http://ow.ly/H06Aj
Feeling the midday slump? Power through it (and avoid it tomorrow) with these 3 tips from @WholeGreenYou

In [37]:
# -------------------------------------------------
frame.head()

,tweetid,date,title,url
0,586278450392133633,Thu Apr 09 21:24:09 +0000 2015,Planning to hire a personal trainer? Read thes...,http://ow.ly/LpxFq
1,586260156155043843,Thu Apr 09 20:11:28 +0000 2015,RT @AnnaMedaris: Any dads out their who strugg...,
2,586248551811932160,Thu Apr 09 19:25:21 +0000 2015,America's problem with diabetes in one map: ...,http://ow.ly/LoXQG
3,586229697165586432,Thu Apr 09 18:10:26 +0000 2015,Think water &amp; fiber will cure your constip...,http://ow.ly/LoBLH
4,586215972731822080,Thu Apr 09 17:15:53 +0000 2015,"About to lose it? Here, try one of these offic...",http://ow.ly/LoBkH


In [38]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Planning to hire a personal trainer? Read these 7 tips first: '

In [39]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class="article-body"]/p/text()',
               '',
               '//*[@id="post-22962"]/header/div[2]/p/span/a/strong/text()'
               ]

In [40]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [41]:
# -------------------------------------------------
# Run for this jupyter notebook
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):

    try:
        # Start the connection
        res = requests.get(url)

        # Get return content as text
        text_html = res.text

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            
            # Initialize XPath
            xpath_selector = Selector(text=text_html)

#             print("Before Scrapting ")
            # Scrapt content with XPath
            list_scrapt = scrapt_w_xpath(list_scrapt)
            print(list_scrapt)

            count_xp_scrapt = len(list_scrapt)
         
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_scrapt.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)
                
                title_attempt = False

            for k_, v_ in list_scrapt.items():
#                 print("v_:", v_)
                frame[k_].iloc[key] = v_
            
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
#     print("----------------------")
    except:
        print(url)
        pass
    

['ERRORS'] 403 , at http://ow.ly/LpxFq

['ERRORS'] 403 , at http://ow.ly/LoXQG
['ERRORS'] 403 , at http://ow.ly/LoBLH
['ERRORS'] 403 , at http://ow.ly/LoBkH
['ERRORS'] 403 , at http://ow.ly/Loypz
['ERRORS'] 403 , at http://ow.ly/Lownl
['ERRORS'] 403 , at http://ow.ly/Lod4r
['ERRORS'] 403 , at http://ow.ly/Lo7Eu
['ERRORS'] 403 , at http://ow.ly/LmFLs
['ERRORS'] 403 , at http://ow.ly/LmFzN
['ERRORS'] 403 , at http://ow.ly/LmFle
['ERRORS'] 403 , at http://ow.ly/LmEMh
['ERRORS'] 403 , at http://ow.ly/LmsCH
['ERRORS'] 403 , at http://ow.ly/Llrhj
['ERRORS'] 403 , at http://ow.ly/LllTo
['ERRORS'] 403 , at http://ow.ly/Lllir
['ERRORS'] 403 , at http://ow.ly/LljLG
['ERRORS'] 403 , at http://ow.ly/LliwN
['ERRORS'] 403 , at http://ow.ly/Llh69
['ERRORS'] 403 , at http://ow.ly/LjDmu
['ERRORS'] 403 , at http://ow.ly/LjCNT
['ERRORS'] 403 , at http://ow.ly/LjD5e
['ERRORS'] 403 , at http://ow.ly/LjcXE
['ERRORS'] 403 , at http://ow.ly/Ljc4A
['ERRORS'] 403 , at http://ow.ly/LjbHS
['ERRORS'] 403 , at http

['ERRORS'] 403 , at http://ow.ly/KDZnD
['ERRORS'] 403 , at http://ow.ly/KDYWK
['ERRORS'] 403 , at http://ow.ly/KDYSu
['ERRORS'] 403 , at http://ow.ly/KCxx0
['ERRORS'] 403 , at http://ow.ly/KCxk4
['ERRORS'] 403 , at http://ow.ly/KCx2q
['ERRORS'] 403 , at http://ow.ly/KBmYf
['ERRORS'] 403 , at http://ow.ly/KCwPt
['ERRORS'] 403 , at http://ow.ly/KCwmB
['ERRORS'] 403 , at http://ow.ly/KCwfl
['ERRORS'] 403 , at http://ow.ly/KBmqK
['ERRORS'] 403 , at http://ow.ly/KBlej
['ERRORS'] 403 , at http://ow.ly/KBm92
['ERRORS'] 403 , at http://ow.ly/KBlzV
['ERRORS'] 403 , at http://ow.ly/KBwQD
['ERRORS'] 403 , at http://ow.ly/KBkwB
['ERRORS'] 403 , at http://ow.ly/KBjP7
['ERRORS'] 403 , at http://bit.ly/19F6i1w
['ERRORS'] 403 , at http://ow.ly/KB3tQ
['ERRORS'] 403 , at http://ow.ly/KAlA2
['ERRORS'] 403 , at http://ow.ly/KA8Fv
['ERRORS'] 403 , at http://ow.ly/KA7M8
['ERRORS'] 403 , at http://ow.ly/KA76z
['ERRORS'] 403 , at http://ow.ly/Kyjvz
['ERRORS'] 403 , at http://ow.ly/Kyjdf

['ERRORS'] 403 , at h

['ERRORS'] 403 , at http://ow.ly/JTgkN
['ERRORS'] 403 , at http://ow.ly/JTg5h
['ERRORS'] 403 , at http://ow.ly/JTfth
['ERRORS'] 403 , at http://ow.ly/JTf07
['ERRORS'] 403 , at http://ow.ly/JS0aU
['ERRORS'] 403 , at http://ow.ly/JSKh3
['ERRORS'] 403 , at http://ow.ly/JS6S6
['ERRORS'] 403 , at http://ow.ly/JSJQr
['ERRORS'] 403 , at http://ow.ly/JRZCS
['ERRORS'] 403 , at http://ow.ly/JS65F
['ERRORS'] 403 , at http://ow.ly/JS2Pb
['ERRORS'] 403 , at http://ow.ly/JRYRp
['ERRORS'] 403 , at http://ow.ly/JQBH3
['ERRORS'] 403 , at http://ow.ly/JQAEp
['ERRORS'] 403 , at http://ow.ly/JQA3N
['ERRORS'] 403 , at http://ow.ly/JQzKR
['ERRORS'] 403 , at http://ow.ly/JQuGD
['ERRORS'] 403 , at http://ow.ly/JQgLO
['ERRORS'] 403 , at http://ow.ly/JQdZB
['ERRORS'] 403 , at http://t.usnews.com/Z6z5tl
['ERRORS'] 403 , at http://ow.ly/JPYSa
['ERRORS'] 403 , at http://ow.ly/JPzxr
['ERRORS'] 403 , at http://ow.ly/JPf4O
['ERRORS'] 403 , at http://t.usnews.com/Z78zpb
['ERRORS'] 403 , at http://ow.ly/JPex9
['ERRORS'

['ERRORS'] 403 , at http://ow.ly/J7WmJ
['ERRORS'] 403 , at http://ow.ly/J7UfV
['ERRORS'] 403 , at http://ow.ly/J7RL1
['ERRORS'] 403 , at http://ow.ly/J7Ota
['ERRORS'] 403 , at http://ow.ly/J5qba
['ERRORS'] 403 , at http://ow.ly/J4bqB
['ERRORS'] 403 , at http://t.usnews.com/Z6q465
['ERRORS'] 403 , at http://ow.ly/J3Qpc
['ERRORS'] 403 , at http://t.usnews.com/Z6qpia
['ERRORS'] 403 , at http://ow.ly/J3Qhb
['ERRORS'] 403 , at http://ow.ly/J3QcC
['ERRORS'] 403 , at http://ow.ly/J2hqU
['ERRORS'] 403 , at http://ow.ly/J2hU5
['ERRORS'] 403 , at http://ow.ly/J2hDi
['ERRORS'] 403 , at http://ow.ly/J2gIp
['ERRORS'] 403 , at http://ow.ly/J2fYa
['ERRORS'] 403 , at http://ow.ly/J2eVn
['ERRORS'] 403 , at http://ow.ly/J2er3
['ERRORS'] 403 , at http://bit.ly/1FHPD7u
['ERRORS'] 403 , at http://ow.ly/J2bs1
['ERRORS'] 403 , at http://ow.ly/J1It0
['ERRORS'] 403 , at http://ow.ly/J0he8
['ERRORS'] 403 , at http://ow.ly/J0hu6
['ERRORS'] 403 , at http://ow.ly/J0vtZ

['ERRORS'] 403 , at http://ow.ly/J0gu9
['ERR

['ERRORS'] 403 , at http://ow.ly/IkMf4
['ERRORS'] 403 , at http://ow.ly/IkLuB
['ERRORS'] 403 , at http://ow.ly/IkL6x
['ERRORS'] 403 , at http://ow.ly/Ii4FE
['ERRORS'] 403 , at http://ow.ly/Ii4rw
['ERRORS'] 403 , at http://ow.ly/Ii3BC
['ERRORS'] 403 , at http://ow.ly/Ii3hW
['ERRORS'] 403 , at http://ow.ly/Ii0Tw
['ERRORS'] 403 , at http://ow.ly/Ii2Ku
['ERRORS'] 403 , at http://spr.ly/6019Isnx
['ERRORS'] 403 , at http://ow.ly/Ii0yE
['ERRORS'] 403 , at http://ow.ly/Ii0pt
['ERRORS'] 403 , at http://ow.ly/IhZXB
['ERRORS'] 403 , at http://ow.ly/IgyZK
['ERRORS'] 403 , at http://ow.ly/IgyxA
['ERRORS'] 403 , at http://ow.ly/I9IOO
['ERRORS'] 403 , at http://ow.ly/IeUWC
['ERRORS'] 403 , at http://ow.ly/I9HcM
['ERRORS'] 403 , at http://ow.ly/IeVlD
['ERRORS'] 403 , at http://ow.ly/IeSNd
['ERRORS'] 403 , at http://buff.ly/1DeTjMa
['ERRORS'] 403 , at http://ow.ly/Id4wD
['ERRORS'] 403 , at http://ow.ly/Idysg
['ERRORS'] 403 , at http://ow.ly/Id6Gb
['ERRORS'] 403 , at http://ow.ly/IdxL1
['ERRORS'] 403 , 

['ERRORS'] 403 , at http://ow.ly/HqiHH
['ERRORS'] 403 , at http://ow.ly/HqffE
['ERRORS'] 403 , at http://ow.ly/HqgWV
['ERRORS'] 403 , at http://ow.ly/Hqgck
['ERRORS'] 403 , at http://ow.ly/HqdPL
['ERRORS'] 403 , at http://bit.ly/1CsArJE
['ERRORS'] 403 , at http://ow.ly/HoSu3
['ERRORS'] 403 , at http://ow.ly/HnV5w
['ERRORS'] 403 , at http://ow.ly/HnRYA
['ERRORS'] 403 , at http://ow.ly/HnsPx
['ERRORS'] 403 , at http://ow.ly/HnUCo
['ERRORS'] 403 , at http://ow.ly/HnQwH
['ERRORS'] 403 , at http://ow.ly/HnrrO
['ERRORS'] 403 , at http://ow.ly/HnJOe
['ERRORS'] 403 , at http://ow.ly/Hnrz6
['ERRORS'] 403 , at http://ow.ly/HnteG
['ERRORS'] 403 , at http://ow.ly/Hnr8E
['ERRORS'] 403 , at http://ow.ly/HmGiu
['ERRORS'] 403 , at http://ow.ly/HmFJo
['ERRORS'] 403 , at http://ow.ly/HmEvc
['ERRORS'] 403 , at http://ow.ly/HmC72
['ERRORS'] 403 , at http://ow.ly/HkBEo
['ERRORS'] 403 , at http://ow.ly/Hkzi4
['ERRORS'] 403 , at http://ow.ly/HkBb3
['ERRORS'] 403 , at http://ow.ly/Hl5UN
['ERRORS'] 403 , at ht

['ERRORS'] 403 , at http://ow.ly/GKcIG
['ERRORS'] 403 , at http://ow.ly/GJZ2x
['ERRORS'] 403 , at http://ow.ly/GJLay
['ERRORS'] 403 , at http://ow.ly/GIESZ
['ERRORS'] 403 , at http://ow.ly/GIEH2
['ERRORS'] 403 , at http://ow.ly/GIEta
['ERRORS'] 403 , at http://ow.ly/GIEno
['ERRORS'] 403 , at http://ow.ly/GIEe6
['ERRORS'] 403 , at http://ow.ly/GIdvg

['ERRORS'] 403 , at http://t.usnews.com/Z5d2p0

['ERRORS'] 403 , at http://ow.ly/GHPoY
['ERRORS'] 403 , at http://ow.ly/GHpWq
Header:   200
IN SCRAPT W XPath
0
http://t.co/
['ERRORS'] 403 , at http://ow.ly/ESJ6e
['ERRORS'] 403 , at http://t.usnews.com/Z5fn3q
['ERRORS'] 403 , at http://ow.ly/GG1jE
['ERRORS'] 403 , at http://ow.ly/GFQYA
['ERRORS'] 403 , at http://ow.ly/GFHsE
['ERRORS'] 403 , at http://ow.ly/GFv29
['ERRORS'] 403 , at http://ow.ly/GFl8m

['ERRORS'] 403 , at http://ow.ly/GEDcB
['ERRORS'] 403 , at http://ow.ly/GErp9
['ERRORS'] 403 , at http://ow.ly/GDzqw
['ERRORS'] 403 , at http://ow.ly/GCxhW
['ERRORS'] 403 , at http://ow.ly/GCpu

In [44]:
# -------------------------------------------------
frame.head()

,tweetid,date,title,url
0,586278450392133633,Thu Apr 09 21:24:09 +0000 2015,Planning to hire a personal trainer? Read thes...,http://ow.ly/LpxFq
1,586260156155043843,Thu Apr 09 20:11:28 +0000 2015,RT @AnnaMedaris: Any dads out their who strugg...,
2,586248551811932160,Thu Apr 09 19:25:21 +0000 2015,America's problem with diabetes in one map: ...,http://ow.ly/LoXQG
3,586229697165586432,Thu Apr 09 18:10:26 +0000 2015,Think water &amp; fiber will cure your constip...,http://ow.ly/LoBLH
4,586215972731822080,Thu Apr 09 17:15:53 +0000 2015,"About to lose it? Here, try one of these offic...",http://ow.ly/LoBkH


In [45]:
# -------------------------------------------------
frame.shape

(1395, 4)